In [2]:
# Import libraries
import pystan # install with pip install pystan
import pandas as pd
import numpy as np
import seaborn as sns
from scipy.stats import beta
import matplotlib.pyplot as plt
import pickle
import hashlib
import time

def get_stan_model(code):
    code_hash = hashlib.sha1(code.encode('utf-8')).hexdigest()
    cache_path =  './cache/' + code_hash + '.pkl'
    try:
        with open(cache_path, 'rb') as file:
            model = pickle.load(file)
    except Exception:
        model = pystan.StanModel(model_code=code) ## c로 변환
        with open(cache_path, 'wb') as file:
            pickle.dump(model, file)
    return model

## 7.1 상호작용

독립변수 간의 교호작용들 등 곱한 항 고려 - 아르바이트 여부가 score가 출석수 y에 미치는 영향의 정도에 영향을 줄때, 위 식을 아래 식으로 변환

In [3]:
#data
df = pd.read_csv('data/data-conc.txt')

#model
model_5_4 ='''
data {
    int T;
    vector [T] Time;
    vector [T] Y;
}

parameters {
    real <lower = 0, upper = 100> a;
    real <lower = 0, upper = 5> b;
    real <lower = 0> sigma;
}

model {
    Y ~ normal(a + b * exp(-b * Time), sigma);
}
        

'''

#python interface

data = {
    'T': df.shape[0],
    'Time' : df.iloc[:,0],
    'Y' : df.iloc[:,1],
}

#result
model_5_4 = get_stan_model(model_5_4)
params = model_5_4.sampling(data = data)
